# Combining prediction from 3 SOTA + literature for training PS model

This is essentially AP prediction + BA prediction from MHCflurry + TAP from DeepTAP + Bashirova for TD

1. The training data for PS is S6 dataset

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [14]:
# read PG + BA data
mhcflurry_df = pd.read_csv('./results/PS_training_MHCflurry_result.csv',index_col=0)
mhcflurry_df['BA_proba'] = [1-np.log(affinity)/np.log(50000) for affinity in mhcflurry_df['affinity']]
mhcflurry_df['BA_y'] = [1 if affinity<500 else 0 for affinity in mhcflurry_df['affinity']]

mhcflurry_df = mhcflurry_df.rename(columns={'processing_score':'PG_proba', 'presentation_score':'PS_proba'})
mhcflurry_df['PG_y'] = [1 if proba>0.5 else 0 for proba in mhcflurry_df['PG_proba']]

# read TAP data
deeptap_df = pd.read_csv('./results/PS_peptides_DeepTAP_cla_predresult.csv')
deeptap_df = deeptap_df.rename(columns={'pred_label':'TAP_y', 'pred_score':'TAP_proba'})

# read TD data
bashirova_df = pd.read_csv('./../../data/TD/processed_data//TD_full.csv',index_col=0)
bashirova_df = bashirova_df.rename(columns={'HLA_full':'allele'})
bashirova_df['TD_y'] = [1 if i>2 else 0 for i in bashirova_df['MFI_ratio']]


In [15]:
# combine datasets
PS_training_df = mhcflurry_df.merge(deeptap_df, on='peptide').merge(bashirova_df, on='allele')
PS_training_df.to_csv('./results/PS_training.csv')
PS_training_df

,peptide,peptide_num,sample_name,affinity,allele,PG_proba,PS_proba,presentation_percentile,PS_y,BA_proba,...,TAP_proba,TAP_y,HLA,MFI_ratio,SD,Source,ID,Sequence,length,TD_y
0,AAAAAAAAAE,13269,sample1,24912.726279,HLA-C*07:01,0.008204,0.004151,62.744674,0,0.064386,...,0.4152,0,C*07:01,1.38,0.45,Bashirova,HLA:HLA00433,MRVMAPRALLLLLSGGLALTETWACSHSMRYFDTAVSRPGRGEPRF...,366,0
1,AAAQFTKLR,11747,sample1,14617.001321,HLA-C*07:01,0.275157,0.017675,13.500897,0,0.113666,...,0.9578,1,C*07:01,1.38,0.45,Bashirova,HLA:HLA00433,MRVMAPRALLLLLSGGLALTETWACSHSMRYFDTAVSRPGRGEPRF...,366,0
2,AAEHSAQVGDP,2430,sample1,28223.131061,HLA-C*07:01,0.004377,0.003630,99.286603,0,0.052855,...,0.3356,0,C*07:01,1.38,0.45,Bashirova,HLA:HLA00433,MRVMAPRALLLLLSGGLALTETWACSHSMRYFDTAVSRPGRGEPRF...,366,0
3,AAGGALGP,13285,sample1,28448.045100,HLA-C*07:01,0.005589,0.003617,99.286603,0,0.052122,...,0.7114,1,C*07:01,1.38,0.45,Bashirova,HLA:HLA00433,MRVMAPRALLLLLSGGLALTETWACSHSMRYFDTAVSRPGRGEPRF...,366,0
4,AAKEWEMW,14316,sample1,23971.646142,HLA-C*07:01,0.292726,0.011703,20.232255,0,0.067945,...,0.9772,1,C*07:01,1.38,0.45,Bashirova,HLA:HLA00433,MRVMAPRALLLLLSGGLALTETWACSHSMRYFDTAVSRPGRGEPRF...,366,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70654,WLTPFLQAVY,560,sample1,10617.317775,HLA-A*26:01,0.270236,0.023554,10.231984,0,0.143213,...,0.6449,1,A*26:01,8.33,2.14,Bashirova,HLA:HLA00073,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFYTSVSRPGRGEPRF...,365,1
70655,YPIELHGIGKY,2175,sample1,10936.552986,HLA-A*26:01,0.773798,0.122202,2.543641,1,0.140475,...,0.3940,0,A*26:01,8.33,2.14,Bashirova,HLA:HLA00073,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFYTSVSRPGRGEPRF...,365,1
70656,YVFGGTTGY,1122,sample1,24.690406,HLA-A*26:01,0.642856,0.970225,0.018723,1,0.703652,...,0.9273,1,A*26:01,8.33,2.14,Bashirova,HLA:HLA00073,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFYTSVSRPGRGEPRF...,365,1
70657,YVIGRRKSY,2400,sample1,117.123267,HLA-A*26:01,0.442252,0.779350,0.310707,1,0.559767,...,0.8421,1,A*26:01,8.33,2.14,Bashirova,HLA:HLA00073,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFYTSVSRPGRGEPRF...,365,1
